## Scaled Dot Product Attention

In [ ]:
import numpy as np

# Step 1: Compute dot product of query and keys
def compute_scores(Q, K):
    return np.dot(Q, K.T)

# Step 2: Scale the scores by the square root of the dimension of keys
def scale_scores(scores, d_k):
    return scores / np.sqrt(d_k)

# Step 3: Apply softmax function to the scores
def softmax(scores):
    exp_scores = np.exp(scores - np.max(scores))  # Numerical stability trick
    return exp_scores / np.sum(exp_scores, axis=-1, keepdims=True)

# Step 4: Compute the weighted sum of the values
def compute_attention_output(attention_weights, V):
    return np.dot(attention_weights, V)

# Main attention function
def attention(Q, K, V):
    d_k = Q.shape[-1]  # Assuming query and key have same dimension
    scores = compute_scores(Q, K)
    scaled_scores = scale_scores(scores, d_k)
    attention_weights = softmax(scaled_scores)
    output = compute_attention_output(attention_weights, V)
    return output, attention_weights

# Example input: query, keys, and values
Q = np.array([[1, 0]])  # Query
K = np.array([[1, 0], [0, 1], [1, 1]])  # Keys
V = np.array([[1, 1], [0, 1], [1, 0]])  # Values

# Apply attention
output, attention_weights = attention(Q, K, V)

print("Output of Attention:", output)
print("Attention Weights:", attention_weights)

Output of Attention: [[0.80222419 0.59888791]]
Attention Weights: [[0.40111209 0.19777581 0.40111209]]


## Text Classification

In [ ]:
import tensorflow as tf
import numpy as np

# Sample data
sentences = [
    'I love programming',
    'I hate bugs',
    'bugs are annoying',
    'programming is awesome',
    'I enjoy coding',
    'I despise errors',
    'errors make me sad',
    'coding makes me happy'
]
labels = [1, 0, 0, 1, 1, 0, 0, 1]  # 1 = Positive, 0 = Negative

# Build vocabulary
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=True, oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1  # Add 1 for padding

# Convert to sequences and pad
sequences = tokenizer.texts_to_sequences(sentences)
X = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding='post')
y = np.array(labels)

# Parameters
embedding_dim = 16
hidden_dim = 16
output_dim = 2
maxlen = X.shape[1]

# Self-Attention layer
class SelfAttention(tf.keras.layers.Layer):
    def __init__(self, hidden_dim):
        super(SelfAttention, self).__init__()
        self.W_q = tf.keras.layers.Dense(hidden_dim)
        self.W_k = tf.keras.layers.Dense(hidden_dim)
        self.W_v = tf.keras.layers.Dense(hidden_dim)

    def call(self, inputs):
        Q = self.W_q(inputs)
        K = self.W_k(inputs)
        V = self.W_v(inputs)
        scores = tf.matmul(Q, K, transpose_b=True)
        dim_k = tf.cast(tf.shape(K)[-1], tf.float32)
        scaled_scores = scores / tf.math.sqrt(dim_k)
        attention_weights = tf.nn.softmax(scaled_scores, axis=-1)
        context = tf.matmul(attention_weights, V)
        return context

# Build model
inputs = tf.keras.Input(shape=(maxlen,))
x = tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)
x = SelfAttention(hidden_dim)(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)  # ✅ Fixed
x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(output_dim, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Train
model.fit(X, y, epochs=30, verbose=1)

# Prediction function
def predict_sentiment(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=maxlen, padding='post')
    probs = model.predict(padded)[0]
    label = np.argmax(probs)
    sentiment = "Positive 😊" if label == 1 else "Negative 😒"
    print(f"Input: {text}")
    print(f"Prediction: {sentiment} | Probabilities: {probs}")

# Test predictions
predict_sentiment("I love coding")
predict_sentiment("bugs make me cry")
predict_sentiment("programming is fun")
predict_sentiment("I hate syntax errors")


Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6250 - loss: 0.6921
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.7500 - loss: 0.6898
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.8750 - loss: 0.6880
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.8750 - loss: 0.6862
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.8750 - loss: 0.6846
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.8750 - loss: 0.6829
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 1.0000 - loss: 0.6812
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 1.0000 - loss: 0.6796
Epoch 9/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 1.0000 - loss: 0.6779
Epoch 10/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 1.0000 - loss: 0.6761
Epoch 11/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 1.0000 - loss: 0.6740
Epoch 12/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 1.0000 - loss: 0.6718
Epo